In [1]:
from pybacktestchain.data_module import FirstTwoMoments
from pybacktestchain.broker import Backtest, StopLoss
from pybacktestchain.blockchain import load_blockchain
from datetime import datetime
import polars as pl
import plotly.express as px

In [2]:
import requests

headers = {
    'authority': 'api.nasdaq.com',
    'accept': 'application/json, text/plain, */*',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Safari/537.36',
    'origin': 'https://www.nasdaq.com',
    'sec-fetch-site': 'same-site',
    'sec-fetch-mode': 'cors',
    'sec-fetch-dest': 'empty',
    'referer': 'https://www.nasdaq.com/',
    'accept-language': 'en-US,en;q=0.9',
}

params = (
    ('tableonly', 'true'),
    ('limit', '25'),
    ('offset', '0'),
    ('download', 'true'),
)

r = requests.get('https://api.nasdaq.com/api/screener/stocks', headers=headers, params=params)
data = r.json()['data']
df = pl.DataFrame(data['rows'])

In [3]:
df.select('country').unique() #.filter(~pl.col('marketCap').eq(""), ~pl.col('ipoyear').eq("")).with_columns(pl.col('volume').cast(pl.Int128), pl.col('marketCap').cast(pl.Float64))

country
str
"""Brazil"""
""""""
"""Canada"""
"""Malaysia"""
"""Italy"""
…
"""Japan"""
"""Turkey"""
"""Philippines"""


In [4]:
df.filter(pl.col('country').is_in(['Mexico'])).select(pl.col('symbol')).to_series().to_list()

['AMX',
 'ASR',
 'BWMX',
 'CX',
 'FMX',
 'KOF',
 'MRNO',
 'MRNOW',
 'OMAB',
 'PAC',
 'SIM',
 'TV',
 'VIST']

In [5]:
# Set verbosity for logging
verbose = False  # Set to True to enable logging, or False to suppress it

backtest = Backtest(
    initial_date=datetime(2019, 1, 1),
    final_date=datetime(2020, 1, 1),
    information_class=FirstTwoMoments,
    risk_model=StopLoss,
    name_blockchain='backtest',
    verbose=verbose,
)

backtest.broker.positions = {}

backtest.universe = df.filter(pl.col('country').is_in(['Mexico'])).select(pl.col('symbol')).to_series().to_list()

backtest.run_backtest()

block_chain = load_blockchain('backtest')


INFO:root:Running backtest from 2019-01-01 00:00:00 to 2020-01-01 00:00:00.
INFO:root:Retrieving price data for universe
ERROR:yfinance:$MRNO: possibly delisted; no price data found  (1d 2019-01-01 -> 2020-01-01) (Yahoo error = "Data doesn't exist for startDate = 1546318800, endDate = 1577854800")
ERROR:yfinance:$MRNOW: possibly delisted; no price data found  (1d 2019-01-01 -> 2020-01-01)
c:\Users\faune\miniconda3\envs\pybacktestchain-gui\Lib\site-packages\pandas\core\frame.py:11211: RuntimeWarning: Degrees of freedom <= 0 for slice
  base_cov = np.cov(mat.T, ddof=ddof)
c:\Users\faune\miniconda3\envs\pybacktestchain-gui\Lib\site-packages\numpy\lib\function_base.py:2748: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)
c:\Users\faune\miniconda3\envs\pybacktestchain-gui\Lib\site-packages\numpy\lib\function_base.py:2748: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)
c:\Users\faune\miniconda3\envs\pybacktestchain-gui

In [6]:
test = (
    pl.read_csv(
        source=fr".\backtests\{block_chain.chain[6].name_backtest}.csv"
    )
    .select(
        pl.col('Date').str.to_date(), 
        pl.exclude('', 'Date')
    )
    .with_columns(dollar_amount = pl.col('Quantity').mul(pl.col('Price')))
    .with_columns(pl.when(pl.col('Action').eq("BUY")).then(-1).otherwise(1).alias('coef'))
    .with_columns(pl.col('dollar_amount').mul(pl.col('coef')).add(1000000))
)

In [7]:
transactions = pl.DataFrame(backtest.broker.get_transaction_log())

In [34]:
(121325*6.669102) - 1000000

-190871.1998500001

In [33]:
transactions.sort('Date')

Date,Action,Ticker,Quantity,Price,Cash
datetime[ns],str,str,i64,f64,f64
2019-01-31 00:00:00,"""BUY""","""BWMX""",121325,6.669102,190871.176565
2019-01-31 00:00:00,"""BUY""","""CX""",36408,5.242317,8.909318
2019-02-28 00:00:00,"""SELL""","""CX""",36408,4.771978,173747.098115
2019-02-28 00:00:00,"""BUY""","""ASR""",3,146.429306,173307.810197
2019-02-28 00:00:00,"""BUY""","""BWMX""",25232,6.711694,3958.341119
…,…,…,…,…,…
2019-11-29 00:00:00,"""BUY""","""KOF""",49,46.42944,22.640109
2019-12-31 00:00:00,"""SELL""","""KOF""",52,48.624664,2551.122653
2019-12-31 00:00:00,"""SELL""","""PAC""",1,101.490585,2652.613238


In [ ]:
px.pie(transactions.join(df.select(pl.exclude('symbol'), pl.col('symbol').alias('Ticker')), on='Ticker').select('sector', 'Quantity').group_by('sector').sum().to_pandas(), values = 'Quantity', names='sector')

In [44]:
px.bar(transactions.sort('Date').select('Ticker', 'Quantity').group_by('Ticker').sum().to_pandas().set_index('Ticker'))

In [31]:
px.scatter(backtest.broker.get_transaction_log().sort_values(by='Date').groupby('Date').last().reset_index()[['Date', 'Cash']].set_index('Date'))

In [28]:
# px.line(backtest.broker.get_transaction_log().sort_values(by='Date').groupby('Date').last().reset_index())